### Dependencias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from scipy import stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import SimpleImputer

from auxiliares import freq, normalizar
import warnings
warnings.filterwarnings('ignore')
cf.go_offline()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Lectura de Datos

In [2]:
train = pd.read_parquet('apptrain.parquet')
valid = pd.read_parquet('appvalid.parquet')
train.shape, valid.shape

((215257, 122), (92254, 121))

In [3]:
buro = pd.read_parquet('buro.parquet')
buro.shape

(1716428, 17)

In [4]:
credit_card = pd.read_parquet('credit_card.parquet')
credit_card.shape

(3840312, 23)

### Universo de trabajo

In [5]:
freq(train,'TARGET')

****Tabla de frecuencias  TARGET  ***


       FA        FR     FAA       FRA
0  197911  0.919417  197911  0.919417
1   17346  0.080583  215257  1.000000






### Pretratamiento de Dataframes

In [6]:
entrenamiento = train['application_id'].tolist()
validacion = valid['application_id'].tolist()
data = validacion + entrenamiento
len(entrenamiento), len(validacion), len(data)

(215257, 92254, 307511)

In [7]:
target = train[['application_id','TARGET']].reset_index(drop=True)
target.shape

(215257, 2)

In [8]:
columnas = train.columns.tolist()[1:]

In [9]:
train=train[columnas]

In [10]:
df = pd.concat([train,valid])
df.shape

(307511, 121)

In [11]:
df['MAX_SCORE']=df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].max(axis = 1)
df['MIN_SCORE']=df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].min(axis = 1)
df['MED_SCORE']=df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].median(axis = 1)

### Caracterización de las variables

In [12]:
varc_1 = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE',
          'DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION',
          'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
          'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
          'DAYS_LAST_PHONE_CHANGE','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
         'MED_SCORE','MIN_SCORE','MAX_SCORE','AMT_REQ_CREDIT_BUREAU_HOUR',
          'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK',
          'AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_YEAR']
vard_1 = ['NAME_CONTRACT_TYPE','CODE_GENDER','NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE','FLAG_MOBIL','FLAG_CONT_MOBILE','OCCUPATION_TYPE',
 'REG_REGION_NOT_LIVE_REGION','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7','FLAG_DOCUMENT_8','FLAG_DOCUMENT_9','FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13','FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']
um_1 = ['application_id']

In [13]:
df = df[varc_1+vard_1+um_1]
df.shape

(307511, 52)

In [14]:
varc_2 = ['DAYS_CREDIT','CREDIT_DAY_OVERDUE','AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_OVERDUE']
vard_2 = ['CREDIT_ACTIVE','CREDIT_TYPE']
um_2 = ['bureau_id']
un_2 =['application_id'] 

In [15]:
buro = buro[um_2+varc_2+vard_2+un_2]
buro.shape

(1716428, 8)

In [16]:
varc_3=['AMT_PAYMENT_TOTAL_CURRENT','AMT_TOTAL_RECEIVABLE','SK_DPD',
        'SK_DPD_DEF']
vard_3=['MONTHS_BALANCE']
un_3=['application_id']

In [17]:
credit_card = credit_card[un_3+varc_3+vard_3]
credit_card.shape

(3840312, 6)

In [18]:
cc= credit_card[credit_card['MONTHS_BALANCE']>=-24].reset_index(drop=True)
cc = cc[un_3+varc_3].groupby(un_3).median().reset_index()
cc.shape

(103558, 5)

In [19]:
df = pd.merge(df,cc, on ='application_id', how ='outer')
df = df[df['application_id'].isin(data)].reset_index(drop=True)
df.shape

(307511, 56)

In [20]:
df = pd.merge(buro,df, on ='application_id', how ='outer')
df = df[df['application_id'].isin(data)].reset_index(drop=True)
df.shape

(1509345, 63)

In [21]:
for d in ('DAYS_CREDIT','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION'):
    df[d]=df[d]/-365.25

In [22]:
df.rename(columns={'DAYS_CREDIT':'YEARS_CREDIT','DAYS_BIRTH':'YEARS_BIRTH',
                   'DAYS_EMPLOYED':'YEARS_EMPLOYED',
                  'DAYS_REGISTRATION':'YEARS_REGISTRATION'},
               inplace=True)

In [23]:
df.head()

,bureau_id,YEARS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_OVERDUE,CREDIT_ACTIVE,CREDIT_TYPE,application_id,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,YEARS_BIRTH,YEARS_EMPLOYED,YEARS_REGISTRATION,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,MED_SCORE,MIN_SCORE,MAX_SCORE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE,CODE_GENDER,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,FLAG_MOBIL,FLAG_CONT_MOBILE,OCCUPATION_TYPE,REG_REGION_NOT_LIVE_REGION,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_PAYMENT_TOTAL_CURRENT,AMT_TOTAL_RECEIVABLE,SK_DPD,SK_DPD_DEF
0,B000315720,1.360712,0.0,0.0,0.0,Closed,Consumer credit,A000198230,270000.0,720000.0,21181.5,720000.0,47.967146,2.570842,3.293634,1.0,1.0,1.0,1.0,-1553.0,0.873736,0.522745,0.231439,0.522745,0.231439,0.873736,0.0,0.0,0.0,0.0,3.0,Cash loans,F,Working,Higher education,1.0,1.0,Accountants,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17887.5,362634.0525,0.0,0.0
1,B001635419,0.569473,0.0,171342.0,0.0,Active,Credit card,A000198230,270000.0,720000.0,21181.5,720000.0,47.967146,2.570842,3.293634,1.0,1.0,1.0,1.0,-1553.0,0.873736,0.522745,0.231439,0.522745,0.231439,0.873736,0.0,0.0,0.0,0.0,3.0,Cash loans,F,Working,Higher education,1.0,1.0,Accountants,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17887.5,362634.0525,0.0,0.0
2,B001059430,0.555784,0.0,NaN,0.0,Active,Consumer credit,A000198230,270000.0,720000.0,21181.5,720000.0,47.967146,2.570842,3.293634,1.0,1.0,1.0,1.0,-1553.0,0.873736,0.522745,0.231439,0.522745,0.231439,0.873736,0.0,0.0,0.0,0.0,3.0,Cash loans,F,Working,Higher education,1.0,1.0,Accountants,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17887.5,362634.0525,0.0,0.0
3,B001464681,0.555784,0.0,NaN,0.0,Active,Credit card,A000198230,270000.0,720000.0,21181.5,720000.0,47.967146,2.570842,3.293634,1.0,1.0,1.0,1.0,-1553.0,0.873736,0.522745,0.231439,0.522745,0.231439,0.873736,0.0,0.0,0.0,0.0,3.0,Cash loans,F,Working,Higher education,1.0,1.0,Accountants,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17887.5,362634.0525,0.0,0.0
4,B000577089,1.722108,0.0,NaN,0.0,Active,Consumer credit,A000198230,270000.0,720000.0,21181.5,720000.0,47.967146,2.570842,3.293634,1.0,1.0,1.0,1.0,-1553.0,0.873736,0.522745,0.231439,0.522745,0.231439,0.873736,0.0,0.0,0.0,0.0,3.0,Cash loans,F,Working,Higher education,1.0,1.0,Accountants,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17887.5,362634.0525,0.0,0.0


In [24]:
varc = varc_1+varc_2+varc_3
vard = vard_1+vard_2
um = um_2+um_1
vart = ['TARGET']

In [25]:
varc =['YEARS_CREDIT' if x=='DAYS_CREDIT' else 'YEARS_BIRTH' if x=='DAYS_BIRTH' 
                   else 'YEARS_EMPLOYED' if x=='DAYS_EMPLOYED'
                  else 'YEARS_REGISTRATION' if x=='DAYS_REGISTRATION' else x for x in varc]

### Análisis Exploratorio de Datos

#### Discreto

##### Ausentes

In [26]:
for v in vard:
    df[v] = df[v].fillna('MISSING').astype(str)

In [27]:
freq(df,vard)

****Tabla de frecuencias  NAME_CONTRACT_TYPE  ***


                      FA        FR      FAA       FRA
Cash loans       1379250  0.913807  1379250  0.913807
Revolving loans   130095  0.086193  1509345  1.000000




****Tabla de frecuencias  CODE_GENDER  ***


          FA        FR      FAA       FRA
F    1029160  0.681859  1029160  0.681859
M     480151  0.318119  1509311  0.999977
XNA       34  0.000023  1509345  1.000000




****Tabla de frecuencias  NAME_INCOME_TYPE  ***


                          FA        FR      FAA       FRA
Businessman               20  0.000013       20  0.000013
Commercial associate  346632  0.229657   346652  0.229670
Maternity leave            5  0.000003   346657  0.229674
Pensioner             265085  0.175629   611742  0.405303
State servant         125512  0.083157   737254  0.488460
Student                   67  0.000044   737321  0.488504
Unemployed                33  0.000022   737354  0.488526
Working               771991  0.511474  1509345  1.

##### Normalización de variables discretas

In [28]:
mapa_norm = list (map(lambda v:normalizar(df,v),vard))


In [29]:
for v,mapa in mapa_norm:
    df[f'n_{v}'] = df[v].replace(mapa)


In [30]:
var_norm = [f'n_{v}' for v in vard]

In [31]:
freq(df,var_norm)

****Tabla de frecuencias  n_NAME_CONTRACT_TYPE  ***


                      FA        FR      FAA       FRA
Cash loans       1379250  0.913807  1379250  0.913807
Revolving loans   130095  0.086193  1509345  1.000000




****Tabla de frecuencias  n_CODE_GENDER  ***


        FA        FR      FAA       FRA
F  1029194  0.681881  1029194  0.681881
M   480151  0.318119  1509345  1.000000




****Tabla de frecuencias  n_NAME_INCOME_TYPE  ***


                          FA        FR      FAA       FRA
Commercial associate  346632  0.229657   346632  0.229657
Pensioner             265085  0.175629   611717  0.405286
State servant         125512  0.083157   737229  0.488443
Working               772116  0.511557  1509345  1.000000




****Tabla de frecuencias  n_NAME_EDUCATION_TYPE  ***


                                    FA        FR      FAA       FRA
Higher education                405270  0.268507   405270  0.268507
Secondary / secondary special  1104075  0.731493  1509345  1.000000






##### Unarias 

In [32]:
unarias = [v for v in var_norm if df[v].nunique()==1]
var_norm = [v for v in var_norm if v not in unarias]
df.drop(unarias,axis=1,inplace=True)

#### Continuas

##### Ausentes

In [33]:
miss = pd.Series(df[varc].isnull().sum()/len(df)).sort_values(ascending=False).to_frame()
miss.columns = ['missing']
fuera = list(miss[miss['missing']>0.3].index)
varc = [v for v in varc if v not in fuera]
df = df.drop(fuera, axis=1)

In [34]:
im = SimpleImputer(strategy='median')
im.fit(df[varc])
df[varc] = im.transform(df[varc])

##### Discretización de variables continuas

In [35]:
kb = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
kb.fit(df[varc])

KBinsDiscretizer(encode='ordinal')

In [36]:
var_disc = [f'disc_{v}' for v in varc]

In [37]:
df[var_disc] = kb.transform(df[varc]).astype(int)

In [38]:
list(map(len,kb.bin_edges_))

[6, 6, 6, 6, 6, 6, 6, 4, 2, 4, 2, 6, 6, 6, 6, 6, 6, 2, 2, 2, 3, 5, 6, 2, 4, 2]

In [39]:
lsta = map(lambda z: dict(enumerate([f'({t[0]}|{t[1]}]'for t in zip(map(str, z), 
            map(str, z[1:]))])), kb.bin_edges_)

In [40]:
for v, d in zip(var_disc, lsta):
    df[v] = df[v].replace(d)

In [41]:
freq(df,var_disc)

****Tabla de frecuencias  disc_AMT_INCOME_TOTAL  ***


                            FA        FR      FAA       FRA
(112500.0|135000.0]     200278  0.132692   200278  0.132692
(135000.0|180000.0]     377657  0.250213   577935  0.382905
(180000.0|225000.0]     258569  0.171312   836504  0.554217
(225000.0|117000000.0]  391361  0.259292  1227865  0.813509
(25650.0|112500.0]      281480  0.186491  1509345  1.000000




****Tabla de frecuencias  disc_AMT_CREDIT  ***


                          FA        FR      FAA       FRA
(263686.5|450000.0]   276407  0.183130   276407  0.183130
(45000.0|263686.5]    301065  0.199467   577472  0.382598
(450000.0|651600.0]   328014  0.217322   905486  0.599920
(651600.0|924394.5]   301889  0.200013  1207375  0.799933
(924394.5|4050000.0]  301970  0.200067  1509345  1.000000




****Tabla de frecuencias  disc_AMT_ANNUITY  ***


                        FA        FR      FAA       FRA
(15192.0|22050.0]   300224  0.198910   300224  0.198910
(1615.5|15192.0]  

##### Unarias

In [42]:
unas = [v for v in var_disc if df[v].nunique()==1]
var_disc = [v for v in var_disc if v not in unas]
df.drop(unas,axis=1,inplace=True)

### Partición

In [43]:
df = df[um+var_norm+var_disc]

In [44]:
train = pd.merge(df[df['application_id'].isin(entrenamiento)].reset_index(drop=True),target,
              on='application_id')
valid = df[df['application_id'].isin(validacion)].reset_index(drop=True)
train.shape,valid.shape

((1057619, 32), (451726, 31))

### Transformación WoE

In [45]:
def transformarWoE(v:str)->dict:
    aux = train[[v]+vart].assign(n=1)
    piv = aux.pivot_table(index=v,columns=vart,values='n',aggfunc='sum',fill_value=0)
    piv/=piv.sum()
    piv['woe'] = np.log(piv[0]/piv[1])
    piv = piv[['woe']]
    return v,piv.to_dict()['woe']

In [46]:
mapa_woe = list(map(transformarWoE,var_norm+var_disc))

In [47]:
for v,mapa in mapa_woe:
    train[f'woe_{v}'] = train[v].replace(mapa)

In [48]:
var_woe = [f'woe_{v}' for v in var_norm+var_disc]

### Poder predictivo

In [49]:
def Information_Value(v:str)->dict:
    aux = train[[v]+vart].assign(n=1)
    piv = aux.pivot_table(index=v,columns=vart,values='n',aggfunc='sum',fill_value=0)
    piv/=piv.sum()
    piv['woe'] = np.log(piv[0]/piv[1])
    piv['iv'] = (piv[0]-piv[1])*piv['woe']
    return v,piv.sum()['iv']

In [50]:
IV = pd.DataFrame(map(Information_Value,var_norm+var_disc),columns=['variable','IV']
                  ).sort_values('IV',ascending=False)
IV

,variable,IV
22,disc_MED_SCORE,0.571823
23,disc_MIN_SCORE,0.498268
24,disc_MAX_SCORE,0.385150
21,disc_EXT_SOURCE_3,0.372674
20,disc_EXT_SOURCE_2,0.240923
15,disc_YEARS_EMPLOYED,0.075941
14,disc_YEARS_BIRTH,0.058893
27,disc_YEARS_CREDIT,0.056488
4,n_OCCUPATION_TYPE,0.043792
3,n_NAME_EDUCATION_TYPE,0.039036


In [51]:
best = list(IV[IV['IV']>0.05]['variable'])
best = [f'woe_{v}' for v in best]
len(best)

8

### Aprendizaje (Regresión Logística)

In [52]:
mod = LogisticRegression()
mod.fit(train[best],train['TARGET'])

LogisticRegression()

In [53]:
coef = mod.coef_
coef

array([[-0.29476059, -0.39691257, -0.27634759, -0.36750029, -0.19100997,
        -0.55361571, -0.0222169 , -0.29261111]])

In [54]:
alpha = mod.intercept_[0]
alpha

-2.457317509792675

In [55]:
roc_auc_score(train['TARGET'],mod.predict_proba(train[best])[:,1])

0.7285195927116526

In [56]:
entrenar = train[um_1+best+vart].groupby('application_id').max().reset_index()

In [57]:
roc_auc_score(entrenar['TARGET'],mod.predict_proba(entrenar[best])[:,1])

0.7212077992752599

### Validación del modelo

In [58]:
for v,mapa in mapa_woe:
    valid[f'woe_{v}'] = valid[v].replace(mapa)

In [59]:
probabilidad = valid[um_1+best]
probabilidad[['prob1','prob']] = mod.predict_proba(probabilidad[best])
probabilidad = probabilidad[['application_id','prob']]
probabilidad=probabilidad[['application_id','prob']].groupby('application_id').max().reset_index() 
probabilidad

,application_id,prob
0,A000000004,0.128299
1,A000000008,0.076998
2,A000000011,0.024135
3,A000000013,0.024113
4,A000000014,0.023972
...,...,...
92249,A000456056,0.027516
92250,A000456060,0.039380
92251,A000456069,0.015789
92252,A000456075,0.020046


In [60]:
probabilidad.to_csv('MARTINEZ_SEDAS_MARIA_GUADALUPE.csv', header = False, index=False, sep=',')